In [ ]:
from IPython.core.display import display, HTML

display(HTML('''<style>
.container {width:98% !important;}
.dataframe th{font: bold 14px times; background: #0ea; text-align: right;}
.dataframe td{font: 14px courier; background: #fff; text-align: right;}
.output_subarea.output_text.output_stream.output_stderr {background: #fff; font-style: italic;}
</style>'''))

In [ ]:
try:
    run_once
except NameError:
    run_once = False
if not run_once:
    run_once = True
    
    import time
    import logging
    reload(logging)
    FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    log_path = 'GEE.log'
    print "logging to %s" % log_path
    logging.basicConfig(filename=log_path,level=logging.DEBUG, format=FORMAT)
    logger = logging.getLogger()
    #logger.basicConfig(filename='/notebooks/Export Microbiome to database.log',level=logging.DEBUG)
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # add formatter to ch
    ch.setFormatter(formatter)

    # add ch to logger
    logger.addHandler(ch)

In [ ]:
%matplotlib inline
import pandas, pandas.io
import re
import seaborn as sns
import math
import scipy, scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import string
import os, os.path
logging.getLogger('boto').setLevel(logging.INFO)
logging.getLogger('p100').setLevel(logging.INFO)
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
PART_DF = '/home/jovyan/work/data/participant_data.pkl'
DATA_DIR = '/home/jovyan/work/data/GEE'

## Setting up quadrants(sic)

In [ ]:
# analytes to apply longitudinal GEE
analytes = [
'total_cholesterol',
'cholesterol_total_quest',
'ldl_cholesterol',
'hdl_cholesterol',
'triglycerides',
'triglycerides_quest',
'ldl_pattern_quest',
'ldl_particle',
'small_ldl_particle',
'ldl_particle_number_quest',
'ldl_medium_quest',
'ldl_small_quest',
'hdl_large_quest',
'glucose',
'glucose_quest',
'hba1c',
'insulin',
'homa_ir',
'interleukin_il6',
'interleukin_il8',
'tnfalpha',
'hs_crp',
'vitamin_d',
'glutathione',
'ferritin_quest',
'zinc',
'methylmalonic_acid',
'methylmalonic_acid_quest',
'selenium',
'mercury',
'copper',
'manganese',
'arachidonic_acid',
'eicosapentaenoic_acid',
'docosapentaenoic_acid',
]

In [ ]:
q_string_inrange = """SELECT co.round, cv.value, cv.chemistry_id, pp.username, cc.name, cc.unit, cr.range_level
                FROM chem_values cv, chem_observations co, chem_chemistries cc, prt_participant pp,
                chem_ranges cr
                WHERE cc.chemistry_id = cv.chemistry_id
                and co.observation_id = cv.observation_id
                and pp.username = co.username
                and cr.chemistry_id = cc.chemistry_id
                and pp.gender = cr.gender
                and (cr.min_value IS NULL or cr.min_value <= cv.value)
                and (cr.max_value IS NULL or cr.max_value >= cv.value)
                and co.round != 4
                and cc.name = %s"""

from math import log10, floor
def round_sig(x, sig=2):
    if (x is None):
        return None
    if (x<0):
        return -round(abs(x), sig-int(floor(log10((abs(x)))))-1)
    elif (x>0):
        return round(abs(x), sig-int(floor(log10((abs(x)))))-1)
    else:
        return 0.0

def RunGEEModel(data_file, part_file):
    
    # Load the analyte data
    data = pandas.read_pickle(data_file)
    
    
    # Load all participants
    prts = pandas.read_pickle(part_file)
    data = data.merge(prts, on=['username'])
    data.dropna(inplace=True)
    unit = data.iloc[0]['unit']
            
    # Get those out of range at baseline
    data['oor'] = (data['round']==1) & (data['range_level']!='INRANGE')
    
    # Set the OOR for all observations if out of range at baseline
    n_oor = 0
    n_total = 0
    n_ir = 0
    
    for username, rows in data.groupby('username'):
        n_total += 1
        if (rows['oor'].sum() == 1):
            data.loc[data['username']==username, ['oor']] = 1
            n_oor+=1
    n_ir = n_total-n_oor
        
    label1 = 'round'
    label2 = 'C(oor)[T.True]:round'
        
    # Run second GEE model over entire population
    mod2 = smf.gee("value ~ age + C(gender) + C(ancestry) + round", "username", data, cov_struct=sm.cov_struct.Independence())
    res2 = mod2.fit()  
    
    cov2 = res2.cov_params()
    conf_int2 = list(res2.conf_int().loc[label1])
    coef2 = res2.params
        
    if (data['oor'].sum() > 0):

        # Run GEE model
        mod = smf.gee("value ~ age + C(gender) + C(ancestry) + C(oor)*round", "username", data, cov_struct=sm.cov_struct.Independence())
        res = mod.fit()
    
        cov = res.cov_params()
        conf_int = list(res.conf_int().loc[label1])
        coef = res.params
    
        gamma = coef[label1] + coef[label2]
        var_gamma = cov.loc[label1, label1] + cov.loc[label2, label2] + 2*cov.loc[label1, label2]
        std_err_gamma = math.sqrt(var_gamma)
        conf_int_gamma = [gamma - 1.965*std_err_gamma, gamma + 1.965*std_err_gamma]
        z_gamma_squared = gamma**2 / var_gamma
        pvalue_gamma = scipy.stats.chi2.sf(z_gamma_squared, 1)
        z_gamma = math.sqrt(z_gamma_squared)
        print analyte, z_gamma_squared, pvalue_gamma
        
    else:
        #No oor values
        return analyte, unit, n_total, round_sig(coef2[label1]), [round_sig(conf_int2[0]), round_sig(conf_int2[1])], round_sig(res2.pvalues.loc[label1]), None, None, None, None, None, None, None, None
    
    # Get in-range statistics
    return analyte, unit, n_total, round_sig(coef2[label1]), [round_sig(conf_int2[0]), round_sig(conf_int2[1])], round_sig(res2.pvalues.loc[label1]), n_ir, round_sig(coef[label1]), [round_sig(conf_int[0]), round_sig(conf_int[1])], round_sig(res.pvalues.loc[label1]), n_oor, round_sig(float(gamma)), [round_sig(conf_int_gamma[0]), round_sig(conf_int_gamma[1])], round_sig(pvalue_gamma)

In [ ]:
results = []
for analyte in analytes:
    print analyte
    dfile = "%s/%s.pkl" % (DATA_DIR, analyte)
    
    results.append(RunGEEModel(dfile, PART_DF))
    
df = pandas.DataFrame(results, columns=['analyte', 'unit', 'all_N', 'all_coef', 'all_confint', 'all_pvalue', 'ir_N', 'ir_coef', 'ir_confint', 'ir_pvalue', 'oor_N', 'oor_coef', 'oor_confint', 'oor_pvalue'])

In [ ]:
# Print table for manuscript in proper order
df_final = df[['analyte', 'unit', 'oor_N', 'oor_coef', 'oor_confint', 'oor_pvalue', 'all_N', 'all_coef', 'all_confint', 'all_pvalue']]

In [ ]:
pandas.set_option('display.precision', 1)
df_final.sort_values('oor_pvalue', ascending=True)